<a href="https://colab.research.google.com/github/ktyptorio/FM-Data-Playground/blob/main/Combining_Data_from_Different_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Preparation

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

path0 = "datafm22.csv"
path1 = "Division Reputation.csv"
path2 = "Club Reputation.csv"
path3 = "National Point.csv"
path4 = "Nation Code.csv"
path5 = "Contract Expired.xlsx"

dataset = pd.read_csv(path0)
divRep = pd.read_csv(path1)
clubRep = pd.read_csv(path2)
natPoint = pd.read_csv(path3)
natCode = pd.read_csv(path4)
contractExp = pd.read_excel(path5)

In [ ]:
dataset

In [ ]:
divRep

In [ ]:
clubRep

In [ ]:
natPoint.head(10)

In [ ]:
natCode

In [6]:
datafm_1 = dataset.copy()

In [7]:
datafm_1[datafm_1['Transfer Value'] == '0']

,Name,Position,Transfer Value,Preferred Foot,Wage,Style,Best Role,Best Pos,Club,Based,...,Ref,TRO,Sta,Str,Tck,Tea,Tec,Thr,Vis,Wor


In [ ]:
contractExp

#Update Division Reputation

In [10]:
datafm_1 = pd.merge(datafm_1,divRep[['Division','Division Reputation']],how='left', on=['Division'])

In [11]:
for counts, values in enumerate(datafm_1):
  if datafm_1['Division'][counts] == 'No Division':
    datafm_1['Division Reputation'][counts] = 'No Division Reputation'

datafm_1['Division Reputation'].fillna('Poor', inplace=True)

In [ ]:
datafm_1[(datafm_1['Division'] != 'No Division') & (datafm_1['Club'] != 'No Club')]

#Update Club Reputation and Nation of Club

In [19]:
datafm_2 = pd.merge(datafm_1,clubRep[['Club','Club Reputation', 'Nation of Club']],how='left', on=['Club'])

In [ ]:
datafm_2[datafm_2['Nation of Club'].isnull() == True]

In [25]:
natList = natPoint['Nation of Player'].unique()

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
  return SequenceMatcher(None, a, b).ratio()

for i in datafm_2[(datafm_2['Nation of Club'].isnull() == True)].index:
  if datafm_2['Division'][i] == 'No Division':
    datafm_2['Nation of Club'][i] = 'No Nation Club'
    pass

  temp1 = 0
  for j in range(227):
    temp = similar(datafm_2['Division'][i].split(' ')[0],natList[j])
    if temp > temp1:
      temp1 = temp
      temp_nation = natList[j]
  datafm_2['Nation of Club'][i] = temp_nation

In [ ]:
datafm_2

In [ ]:
datafm_2.sort_values(['Division'], inplace=True)
datafm_2['Club Reputation'].fillna(method='ffill', inplace=True)
datafm_2.sort_index(inplace=True)

In [ ]:
datafm_2.info()

#Update Nation of Player

In [36]:
datafm_3 = pd.merge(datafm_2,natCode[['Nation','Nation of Player']],how='left', on=['Nation'])

#Update Nation Reputation and Continent of Player

In [37]:
datafm_4 = pd.merge(datafm_3,natPoint[['Nation of Player','Continent of Player', 'Nation Points']],how='left', on=['Nation of Player'])

In [ ]:
datafm_4.info()

#Update Continent of Club

In [40]:
natPoint_x = natPoint.rename(columns={'Nation of Player':'Nation of Club', 'Continent of Player':'Continent of Club',})

In [41]:
datafm_5 = pd.merge(datafm_4,natPoint_x[['Nation of Club','Continent of Club']],how='left', on=['Nation of Club'])

In [ ]:
datafm_5

#Update Contract Expired

In [77]:
datafm_6 = datafm_5.sort_values(by=['Name']).reset_index()
contractExp = contractExp.sort_values(by=['Name']).reset_index()
datafm_6 = pd.merge(datafm_6,contractExp['Expired Date'], left_index=True, right_index=True )
del datafm_6['index']

In [ ]:
datafm_6